<a href="https://colab.research.google.com/github/tms-byte/signate/blob/main/taitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/tms-byte/signate/blob/main/taitanic.ipynb

In [1]:
!pip install signate

     |████████████████████████████████| 138 kB 8.3 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=1fef7291e2aacfd0af0fb83d8fc101890c793d1cce7fb42ace16eff42ceeacc9
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is i

In [4]:
from os import mkdir
from google.colab import files
upload=files.upload()
# uploadしたsignate.jsonは/contentに保存される。
! mkdir /root/.signate
! pwd
! ls -a /root/
! cp signate.json /root/.signate/
! ls
! ls /root/.signate/

Saving signate.json to signate (1).json
/content
.   .bashrc  .config  .ipython	.keras	.npm	  .signate
..  .cache   .gsutil  .jupyter	.local	.profile  .tmux.conf
 sample_data  'signate (1).json'   signate.json
signate.json


In [5]:
! signate --help

Usage: signate [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  download  Download the file of competition
  files     List file of competition
  list      List competition
  submit    Submit a result file to the...
  token     Download the API Token


In [6]:
!signate list

  competitionId  title                                                                                     closing     prize                  submitters
---------------  ----------------------------------------------------------------------------------------  ----------  -------------------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                                      -                                        5416
             24  【練習問題】お弁当の需要予測                                                              -                                        6700
             27  【練習問題】Jリーグの観客動員数予測                                                       -                                        1576
            100  【練習問題】手書き文字認識                                                                -           Knowledge                     204
            102  【練習問題】タイタニックの生存予測                                                        -           Knowledge                    1278
        

In [7]:
!signate files -c 102

  fileId  name               title                     size  updated_at
--------  -----------------  ----------------------  ------  -------------------
     272  train.tsv          学習用データ             12932  2018-05-19 00:50:34
     273  test.tsv           評価用データ             12052  2018-05-19 00:50:45
     430  sample_submit.tsv  応募用サンプルファイル    3063  2018-07-06 00:14:18


In [8]:
! signate download -c 102

sample_submit.tsv

test.tsv

train.tsv


Download completed.


# colabの前処理は終了。以下は通常通りコードを書く
pd.read_csvメソッドのindex_colはどこの行から読み始めるかの指定。

通常は初めから読みたいから読みたいから0を指定する。

header=None : 初めのヘッダーの省略。データを読み込む際に邪魔

In [9]:
import pandas as pd
train=pd.read_csv("train.tsv",sep="\t",index_col=0)
test=pd.read_csv("test.tsv",sep="\t",index_col=0)
sample_submit=pd.read_csv("sample_submit.tsv",sep="\t",index_col=0,
                          header=None)# 応募用サンプルファイル

In [10]:
# head()メソッドは先頭先頭5行目までしか読めない
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,,
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
7,0,3,male,2.0,3,1,21.0750,S
9,1,2,female,14.0,1,0,30.0708,C
11,1,1,female,58.0,0,0,26.5500,S


# 目的

学習用データにはあった目的変数であるカラムsurvivedが無いことが分かります。その他のカラムは学習用データと同じです。つまり、答えのある学習用データの乗客の情報を元にして機械学習により生存の法則をモデル化し、そのモデルを評価用データに当てはめて、評価用データに記録されている各乗客が生存できるのか否かを予測することが課題になります。

提出するまではテストデータに対する正解率が不明。

提出時に尤度を埋める。

In [11]:
test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
5,3,male,NaN,0,0,8.4583,Q
6,1,male,54.0,0,0,51.8625,S


In [16]:
print(train.shape)
print(test.shape)
print("\n\t改行テスト")
train.info()

(445, 8)
(446, 7)

	改行テスト
<class 'pandas.core.frame.DataFrame'>
Int64Index: 445 entries, 3 to 888
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  445 non-null    int64  
 1   pclass    445 non-null    int64  
 2   sex       445 non-null    object 
 3   age       360 non-null    float64
 4   sibsp     445 non-null    int64  
 5   parch     445 non-null    int64  
 6   fare      445 non-null    float64
 7   embarked  443 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 31.3+ KB


ageを見るとデータ数が360となっており、他のデータよりも少なくなっています。これから、何人かの乗客は年齢や乗船した港の情報が存在しない、つまり値が欠損しているということが分かりました。embarkedも2つ欠損があるようです。評価用データも同様に確認すると、学習用データと同様、ageに欠損値があることが分かります。また、sexとembarkedは、データタイプがobject、つまりカテゴリ変数であることも分かります。

In [20]:
train['survived'].value_counts()
x0,x1=train['survived'].value_counts()
print(x1/(x0+x1)*100)

40.2247191011236


In [18]:
train.describe()

,survived,pclass,age,sibsp,parch,fare
count,445.000000,445.000000,360.000000,445.000000,445.000000,445.000000
mean,0.402247,2.296629,29.211583,0.546067,0.431461,33.959971
std,0.490903,0.834024,14.154300,1.195247,0.850489,52.079492
min,0.000000,1.000000,0.670000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.000000,0.000000,0.000000,7.925000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,15.000000
75%,1.000000,3.000000,37.250000,1.000000,1.000000,31.387500
max,1.000000,3.000000,80.000000,8.000000,5.000000,512.329200


In [21]:
pd.get_dummies(train).corrwith(train["survived"])

survived      1.000000
pclass       -0.358097
age          -0.081394
sibsp        -0.045087
parch         0.079669
fare          0.258605
sex_female    0.559465
sex_male     -0.559465
embarked_C    0.182568
embarked_Q    0.005062
embarked_S   -0.172769
dtype: float64

In [23]:
train[["pclass","survived"]].groupby(["pclass"]).mean()

,survived
pclass,
1,0.685185
2,0.443299
3,0.258333


In [24]:
train[["sex","survived"]].groupby(["sex"]).mean()

,survived
sex,
female,0.775641
male,0.200692


In [25]:
data=pd.concat([train,test])

In [26]:
data["age"]=data["age"].fillna(data["age"].mean())
data["embarked"]=data["embarked"].fillna(data["embarked"].mode())

In [27]:
data=pd.get_dummies(data)

pd.index はid番号を取得

そして、locメソッドでid番号の範囲の行データを取得

勿論勿論trainとtestでデータid番号は異なるから分離可能

In [29]:
train=data.loc[train.index]
test=data.loc[test.index]
# axis=1で列情報を削除
test=test.drop(["survived"],axis=1)
train.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
id,,,,,,,,,,,
3,1.0,1,35.0,1,0,53.1000,1,0,0,0,1
4,0.0,3,35.0,0,0,8.0500,0,1,0,0,1
7,0.0,3,2.0,3,1,21.0750,0,1,0,0,1
9,1.0,2,14.0,1,0,30.0708,1,0,1,0,0
11,1.0,1,58.0,0,0,26.5500,1,0,0,0,1


In [30]:
y=train["survived"] # 目的変数
X=train.drop(["survived"],axis=1) # 目的変数を除いたデータ

In [32]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [33]:
pred=model.predict_proba(test)[:,1]
print(pred[:5])

[0.10554246 0.92164857 0.64522003 0.17226274 0.28096284]


In [34]:
sample_submit[1]=pred
sample_submit.to_csv('submit.tsv', header=None,sep='\t')

In [35]:
! signate submit -c 102 -n "submit from Google Colaboratory" sample_submit.tsv

You have successfully submitted your predictions.We will send you the submission result to your email address.
